<a href="https://colab.research.google.com/github/Vrussel/Mental-Health-Chatbot/blob/main/Mental_Health_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import google.generativeai as genai
from google.colab import userdata
import re # Import regex for input filtering
from transformers import pipeline # Import pipeline for sentiment analysis

# Configure the Gemini API
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
# Initialize the Generative Model
# You can choose a different model if needed
gemini_model = genai.GenerativeModel('gemini-2.5-flash')

# Initialize sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

def mental_health_chatbot(prompt):
  """
  Sends a prompt to the Gemini model and returns the response with ethical guardrails and sentiment analysis.
  """

  # --- Sentiment Analysis ---
  try:
    sentiment = sentiment_analyzer(prompt)[0]
    sentiment_label = sentiment['label']
    sentiment_score = sentiment['score']
    print(f"Sentiment detected: {sentiment_label} with score {sentiment_score:.2f}") # Optional: for debugging/monitoring
  except Exception as e:
    print(f"Sentiment analysis failed: {e}")
    sentiment_label = "neutral" # Default to neutral if analysis fails


  # --- Input Filtering (More Advanced) ---
  # Combine keyword checking with sentiment for potentially better filtering
  # This is a simplified example; more complex logic can be added
  off_topic_keywords = ["sports", "politics", "math", "calculate", "solve", "science", "history", "news", "entertainment", "technology", "finance", "legal"]
  if any(keyword in prompt.lower() for keyword in off_topic_keywords) and sentiment_label == "neutral":
       return "I'm sorry, I can only discuss topics related to health and mental well-being. For other questions, please consult a relevant professional or resource."

  # Redirect severe prompts - keep this relatively strict
  severe_keywords = ["suicidal", "harm myself", "emergency", "crisis", "danger", "abuse"]
  if any(keyword in prompt.lower() for keyword in severe_keywords) or (sentiment_label == "negative" and sentiment_score > 0.9): # Consider strong negative sentiment as a trigger
      return "If you are experiencing a mental health crisis or feel in danger, please seek immediate help. You can contact a crisis hotline, emergency services, or a mental health professional in your area. I am not equipped to handle emergencies."

  # Filter medical diagnosis/medication requests
  medical_keywords = ["medical diagnosis", "medication", "prescription", "diagnose", "cure", "treat"]
  if any(keyword in prompt.lower() for keyword in medical_keywords):
      return "I cannot provide medical diagnoses or recommend medication. Please consult a healthcare professional for personalized medical advice."

  # Add helpline number for mental health issues
  mental_health_keywords = ["depressed", "anxious", "stress", "sad", "lonely", "struggling", "mental health"]
  helpline_number = "National Crisis and Suicide Lifeline: 988"
  if any(keyword in prompt.lower() for keyword in mental_health_keywords) or sentiment_label in ["negative", "very negative"]:
      helpline_message = f"\n\nIf you are experiencing a mental health crisis, please reach out for help. You can contact the {helpline_number}."
  else:
      helpline_message = ""


  # --- Ethical Directions to the LLM (via prompt engineering) ---
  # Add instructions to the prompt to guide the model's behavior
  # Include sentiment information in the prompt to potentially guide the response tone
  ethical_prompt = f"""You are a helpful and supportive mental health chatbot providing general information and support related to health and mental well-being.
  The user's input has been analyzed as having a {sentiment_label} sentiment.
  Strictly adhere to the following guidelines:
  - Your primary focus is to provide helpful and general information about mental health and well-being.
  - Do not provide medical diagnoses, recommend medication, or advise on physical health issues.
  - Do not generate content related to self-harm, harm to others, or illegal activities.
  - If a query is severe (e.g., related to suicidal thoughts, crisis, or danger), immediately and strongly recommend seeking help from a qualified mental health professional, crisis hotline, or emergency services. Do not attempt to handle the crisis yourself.
  - If a query is off-topic (not related to health or mental well-being), politely state that you can only discuss those topics and offer to help with a relevant question.
  - Always prioritize providing accurate and general information within the scope of health and mental well-being.
  - Be liberal in your interpretation of what is "health or mental health related" to be as helpful as possible within the safety guidelines.
  - Respond in a tone appropriate to the detected sentiment, while remaining professional and supportive.
  - Give a thorough response and suggest multiple solutions
  ONLY WRITE 20 WORDS PER LINE

  Respond to the following user input:

  {prompt}
  """

  try:
    response = gemini_model.generate_content(ethical_prompt)
    raw_response_text = response.text.strip()

    # --- Advanced Output Safety ---
    # Use regex or more complex pattern matching for sensitive topics if needed
    # For now, continue with keyword checks and rely on prompt engineering
    sensitive_keywords_output = ["harm", "kill yourself", "suicide methods", "medication", "prescription", "diagnose"]
    if any(keyword in raw_response_text.lower() for keyword in sensitive_keywords_output):
         return "I cannot provide information related to harmful actions, medication, or diagnosis. If you are having thoughts of harming yourself, please seek immediate help from a professional or crisis service."


    # Add disclaimers to all responses
    disclaimer = "\n\nDisclaimer: This information is for general guidance only and does not replace professional advice. Consult a healthcare professional for personalized support."

    # Include a reminder for severe cases if the model's response didn't already strongly recommend it (as a fallback)
    severe_reminder = ""
    if any(keyword in prompt.lower() for keyword in severe_keywords) and not any(keyword in raw_response_text.lower() for keyword in ["crisis hotline", "emergency services", "mental health professional", "seek help"]):
         severe_reminder = "\n\nIf you are experiencing a mental health crisis, please seek immediate help from a professional or crisis service."


    return raw_response_text + helpline_message + severe_reminder + disclaimer

  except Exception as e:
    return f"An error occurred: {e}"

if __name__ == "__main__":
  print("Mental Health Chatbot (Type 'exit' or 'quit' to end)")
  while True:
    user_input = input("You: ")
    if user_input.lower() == "exit" or user_input.lower() == "quit":
      break
    response = mental_health_chatbot(user_input)
    print("Mental Health Chatbot: ", response)

In [ ]:
for m in genai.list_models():
  print(f"{m.name}: {m.supported_generation_methods}")

In [ ]:
from google.colab import userdata
import google.generativeai as genai

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)
print("Gemini API configured successfully.")

In [ ]:

from google.colab import drive
drive.mount('/content/drive')